In [1]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH21-LangGraph")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH21-LangGraph


# 대화기록 요약 추가 

- 대화 기록 유지, 지속성 
    - 대화를 지속하기 쉽게 만들어줌 
    - 하지만 대화가 길어질수록 대화 기록이 누적되어 `context window`를 더 많이 차지하게 됨
    - `LLM` 호출이 더 비싸고 길어지며, 잠재적으로 오류가 발생할 수 있어 바람직하지 않을 수 있음 
    - 이를 해결하기 위한 한 가지 방법은 현재까지의 대화 요약본을 생성하고, 이를 최근 `N` 개의 메시지와 함께 사용하는 것 

- 프로세스
    - 대화가 너무 긴지 확인 (메시지 수나 메시지 길이로 확인 가능)
    - 너무 길다면 요약본 생성 (이를 위한 프롬프트 필요)
    - 마지막 `N` 개의 메시지를 제외한 나머지 삭제 (`DeleteMessage`)

In [3]:
from typing import Literal, Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, RemoveMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.graph.message import add_messages

In [4]:
# 메모리 저장소 설정
memory = MemorySaver()


# 메시지 상태와 요약 정보를 포함하는 상태 클래스
class State(MessagesState):
    messages: Annotated[list, add_messages]
    summary: str


# 모델 초기화
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [5]:
def ask_llm(state: State):
    # 이전 요약 정보 확인
    summary = state.get("summary", "")

    # 이전 요약 정보가 있다면 시스템 메시지로 추가
    if summary:
        system_message = f"Summary of conversation earlier: {summary}"
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else:
        messages = state["messages"]

    # 모델 호출
    response = model.invoke(messages)

    return {"messages": [response]}

In [6]:
# 대화 종료 또는 요약 결정 로직
# Literal : 값의 범위를 제한하고 의도를 명확히 표현할 수 있게 해두는 도구 (Type Hint) 
def should_continue(state: State) -> Literal["summarize_conversation", END]:
    # 메시지 목록 확인
    messages = state["messages"]

    # 메시지 수가 6개 초과라면 요약 노드로 이동
    if len(messages) > 6:
        return "summarize_conversation"
    return END

In [7]:
# 대화 내용 요약 및 메시지 정리 로직
def summarize_conversation(state: State):
    # 이전 요약 정보 확인
    summary = state.get("summary", "")

    # 이전 요약 정보가 있다면 요약 메시지 생성
    if summary:
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above in Korean:"
        )
    else:
        # 요약 메시지 생성
        summary_message = "Create a summary of the conversation above in Korean:"


    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    
    return {"summary": response.content, "messages": delete_messages}

In [8]:
# GRAPH
workflow = StateGraph(State)

workflow.add_node("conversation", ask_llm)
workflow.add_node(summarize_conversation)
workflow.add_edge(START, "conversation")
workflow.add_conditional_edges(
    "conversation",
    should_continue,
)
workflow.add_edge("summarize_conversation", END)

app = workflow.compile(checkpointer=memory)

In [9]:
from langchain_teddynote.graphs import visualize_graph

visualize_graph(app)

[ERROR] Visualize Graph Error: Failed to render the graph using the Mermaid.INK API. Status code: 204.


In [10]:
print(app.get_graph().draw_ascii())

                      +-----------+           
                      | __start__ |           
                      +-----------+           
                             *                
                             *                
                             *                
                     +--------------+         
                     | conversation |         
                     +--------------+         
                    ...             ...       
                  ..                   ...    
                ..                        ..  
+------------------------+                  ..
| summarize_conversation |                ..  
+------------------------+             ...    
                    ***             ...       
                       **         ..          
                         **     ..            
                        +---------+           
                        | __end__ |           
                        +---------+           


**그래프 해석**

- 기본 대화는 항상 `conversation`에서 시작
- 요약이 필요한 조건일 때만 `summarize_conversation` 실행
- 모든 흐름은 결국 `__end__`로 종료됨

In [11]:
# 업데이트 정보 출력 함수
def print_update(update):
    # 업데이트 딕셔너리 순회
    for k, v in update.items():
        # 메시지 목록 출력
        for m in v["messages"]:
            m.pretty_print()
        # 요약 정보 존재 시 출력
        if "summary" in v:
            print(v["summary"])

In [12]:
# 설정 
config = {"configurable": {"thread_id": "001"}}

# 첫 번째 사용자 메시지 생성 및 출력
input_message = HumanMessage(content="안녕하세요? 반갑습니다. 제 이름은 김철수 입니다.")
input_message.pretty_print()

# 스트림 모드에서 첫 번째 메시지 처리 및 업데이트 출력
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

# 두 번째 사용자 메시지 생성 및 출력
input_message = HumanMessage(content="제 이름이 뭔지 기억하세요?")
input_message.pretty_print()

# 스트림 모드에서 두 번째 메시지 처리 및 업데이트 출력
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

# 세 번째 사용자 메시지 생성 및 출력
input_message = HumanMessage(content="제 직업은 AI 엔지니어 입니다.")
input_message.pretty_print()

# 스트림 모드에서 세 번째 메시지 처리 및 업데이트 출력
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

안녕하세요? 반갑습니다. 제 이름은 김철수 입니다.
================================== Ai Message ==================================

안녕하세요, 김철수님! 반갑습니다. 어떻게 도와드릴까요?
================================ Human Message =================================

제 이름이 뭔지 기억하세요?
================================== Ai Message ==================================

네, 김철수님이라고 하셨습니다. 다른 질문이나 이야기하고 싶은 것이 있으신가요?
================================ Human Message =================================

제 직업은 AI 엔지니어 입니다.
================================== Ai Message ==================================

멋진 직업이네요, 김철수님! AI 엔지니어로서 어떤 분야에 주로 집중하고 계신가요? 또는 어떤 프로젝트에 참여하고 계신지 궁금합니다.


In [13]:
# 상태 구성 값 검색
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='안녕하세요? 반갑습니다. 제 이름은 김철수 입니다.', id='4cc47bf6-f83d-4c04-acd6-f2ac9701bc5b'),
  AIMessage(content='안녕하세요, 김철수님! 반갑습니다. 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 22, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'finish_reason': 'stop', 'logprobs': None}, id='run-48f696cc-9a6b-494c-90ef-09811936e4f8-0', usage_metadata={'input_tokens': 22, 'output_tokens': 19, 'total_tokens': 41}),
  HumanMessage(content='제 이름이 뭔지 기억하세요?', id='f0d82e5c-3338-49b5-9813-5dd9486ea9b7'),
  AIMessage(content='네, 김철수님이라고 하셨습니다. 다른 질문이나 이야기하고 싶은 것이 있으신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_t

In [14]:
# 사용자 입력 메시지 객체 생성
input_message = HumanMessage(
    content="2025년 AI 트랜드 에 대해 좀 더 알아보고 있어요. 최근 기사를 읽고 있습니다."
)

# 메시지 내용 출력
input_message.pretty_print()

# 스트림 이벤트 실시간 처리 및 업데이트 출력
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

2025년 AI 트랜드 에 대해 좀 더 알아보고 있어요. 최근 기사를 읽고 있습니다.
================================== Ai Message ==================================

2025년의 AI 트렌드는 여러 가지 흥미로운 방향으로 발전할 것으로 예상됩니다. 몇 가지 주요 트렌드를 소개해드릴게요:

1. **AI의 민주화**: 더 많은 사람들이 AI 기술을 쉽게 접근하고 사용할 수 있도록 하는 도구와 플랫폼이 증가할 것입니다. 이는 비전문가도 AI를 활용할 수 있게 해줄 것입니다.

2. **Explainable AI (XAI)**: AI의 결정 과정을 이해하고 설명할 수 있는 기술이 중요해질 것입니다. 이는 특히 의료, 금융 등 중요한 분야에서 신뢰성을 높이는 데 기여할 것입니다.

3. **AI와 윤리**: AI의 윤리적 사용에 대한 논의가 더욱 활발해질 것입니다. 데이터 프라이버시, 편향성 문제 등을 해결하기 위한 규제와 가이드라인이 필요할 것입니다.

4. **AI와 자동화**: 다양한 산업에서 AI를 활용한 자동화가 증가할 것입니다. 이는 생산성을 높이고 비용을 절감하는 데 기여할 것입니다.

5. **인간-AI 협업**: AI와 인간이 협력하여 문제를 해결하는 방식이 더욱 중요해질 것입니다. AI는 반복적인 작업을 처리하고, 인간은 창의적이고 전략적인 결정을 내리는 역할을 할 것입니다.

6. **AI의 지속 가능성**: 환경 문제 해결을 위한 AI의 역할이 강조될 것입니다. 에너지 효율성을 높이거나 기후 변화에 대응하는 데 AI가 활용될 것입니다.

이 외에도 다양한 트렌드가 있을 수 있으니, 최신 기사를 통해 계속해서 정보를 업데이트하는 것이 좋습니다. 어떤 특정한 주제에 대해 더 알고 싶으신가요?
===========================

/var/folders/37/2rwxb6k520l4tqxzpx1nh8lh0000gn/T/ipykernel_36858/3396824294.py:19: LangChainBetaWarning: The class `RemoveMessage` is in beta. It is actively being worked on, so the API may change.
  delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]


In [15]:
# 상태 구성 값 검색
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='2025년 AI 트랜드 에 대해 좀 더 알아보고 있어요. 최근 기사를 읽고 있습니다.', id='57375dc3-94b3-40e4-aff3-de9ef79695ec'),
  AIMessage(content='2025년의 AI 트렌드는 여러 가지 흥미로운 방향으로 발전할 것으로 예상됩니다. 몇 가지 주요 트렌드를 소개해드릴게요:\n\n1. **AI의 민주화**: 더 많은 사람들이 AI 기술을 쉽게 접근하고 사용할 수 있도록 하는 도구와 플랫폼이 증가할 것입니다. 이는 비전문가도 AI를 활용할 수 있게 해줄 것입니다.\n\n2. **Explainable AI (XAI)**: AI의 결정 과정을 이해하고 설명할 수 있는 기술이 중요해질 것입니다. 이는 특히 의료, 금융 등 중요한 분야에서 신뢰성을 높이는 데 기여할 것입니다.\n\n3. **AI와 윤리**: AI의 윤리적 사용에 대한 논의가 더욱 활발해질 것입니다. 데이터 프라이버시, 편향성 문제 등을 해결하기 위한 규제와 가이드라인이 필요할 것입니다.\n\n4. **AI와 자동화**: 다양한 산업에서 AI를 활용한 자동화가 증가할 것입니다. 이는 생산성을 높이고 비용을 절감하는 데 기여할 것입니다.\n\n5. **인간-AI 협업**: AI와 인간이 협력하여 문제를 해결하는 방식이 더욱 중요해질 것입니다. AI는 반복적인 작업을 처리하고, 인간은 창의적이고 전략적인 결정을 내리는 역할을 할 것입니다.\n\n6. **AI의 지속 가능성**: 환경 문제 해결을 위한 AI의 역할이 강조될 것입니다. 에너지 효율성을 높이거나 기후 변화에 대응하는 데 AI가 활용될 것입니다.\n\n이 외에도 다양한 트렌드가 있을 수 있으니, 최신 기사를 통해 계속해서 정보를 업데이트하는 것이 좋습니다. 어떤 특정한 주제에 대해 더 알고 싶으신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {

In [16]:
messages = values["messages"]
messages

[HumanMessage(content='2025년 AI 트랜드 에 대해 좀 더 알아보고 있어요. 최근 기사를 읽고 있습니다.', id='57375dc3-94b3-40e4-aff3-de9ef79695ec'),
 AIMessage(content='2025년의 AI 트렌드는 여러 가지 흥미로운 방향으로 발전할 것으로 예상됩니다. 몇 가지 주요 트렌드를 소개해드릴게요:\n\n1. **AI의 민주화**: 더 많은 사람들이 AI 기술을 쉽게 접근하고 사용할 수 있도록 하는 도구와 플랫폼이 증가할 것입니다. 이는 비전문가도 AI를 활용할 수 있게 해줄 것입니다.\n\n2. **Explainable AI (XAI)**: AI의 결정 과정을 이해하고 설명할 수 있는 기술이 중요해질 것입니다. 이는 특히 의료, 금융 등 중요한 분야에서 신뢰성을 높이는 데 기여할 것입니다.\n\n3. **AI와 윤리**: AI의 윤리적 사용에 대한 논의가 더욱 활발해질 것입니다. 데이터 프라이버시, 편향성 문제 등을 해결하기 위한 규제와 가이드라인이 필요할 것입니다.\n\n4. **AI와 자동화**: 다양한 산업에서 AI를 활용한 자동화가 증가할 것입니다. 이는 생산성을 높이고 비용을 절감하는 데 기여할 것입니다.\n\n5. **인간-AI 협업**: AI와 인간이 협력하여 문제를 해결하는 방식이 더욱 중요해질 것입니다. AI는 반복적인 작업을 처리하고, 인간은 창의적이고 전략적인 결정을 내리는 역할을 할 것입니다.\n\n6. **AI의 지속 가능성**: 환경 문제 해결을 위한 AI의 역할이 강조될 것입니다. 에너지 효율성을 높이거나 기후 변화에 대응하는 데 AI가 활용될 것입니다.\n\n이 외에도 다양한 트렌드가 있을 수 있으니, 최신 기사를 통해 계속해서 정보를 업데이트하는 것이 좋습니다. 어떤 특정한 주제에 대해 더 알고 싶으신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_to

In [17]:
# 사용자 메시지 객체 생성
input_message = HumanMessage(content="제 이름이 무엇인지 기억하세요?")

# 메시지 내용 출력
input_message.pretty_print()

# 스트림 이벤트 실시간 처리 및 업데이트
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

제 이름이 무엇인지 기억하세요?
================================== Ai Message ==================================

네, 당신의 이름은 김철수님입니다. 도움이 필요하시면 언제든지 말씀해 주세요!


In [18]:
# 사용자 메시지 객체 생성
input_message = HumanMessage(content="제 직업도 혹시 기억하고 계세요?")

# 메시지 내용 출력
input_message.pretty_print()

# 스트림 이벤트 실시간 처리 및 업데이트 출력
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

제 직업도 혹시 기억하고 계세요?
================================== Ai Message ==================================

네, 김철수님은 AI 엔지니어로 일하고 계십니다. AI와 관련된 주제에 대해 더 알고 싶으신 점이 있으면 말씀해 주세요!


-----
** End of Documents **